In [75]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

#!pip install dwave-ocean-sdk
#!pip show dwave-ocean-sdk
#!pip show numpy

import dimod
from dwave.embedding import MinimizeEnergy, embed_bqm
from dwave.system import DWaveSampler
import math
import numpy as np
from pyqubo import Array, Constraint, Placeholder, solve_qubo, Binary
import sys
import re
import neal

def dist_cal(start, end):
    dist = math.sqrt((start[0] - end[0]) ** 2 + (start[1] - end[1]) ** 2)
    return dist

def read_distances(filename):
    tasks = []
    velocity = []
    with open(filename, 'r') as f:
        for line in f:
            # Skip comments
            if line[0] == '#':
                continue
            else:
                l = list(map(int, (line.strip()).split(',')))
                tasks.append(list(l[0:2]))
                tasks.append(list(l[2:4]))
                velocity.append(l[4])
    tasks = np.array(tasks)

    return tasks, velocity

#read problem(infomation of tasks)
#arg = sys.argv[1]
tasks, velocity = read_distances('ex.csv')
n = len(velocity)
row = n
column = n * 2

# creating instance of problem
x = []
for i in range(row * column):
    x.append(Binary('x[{}]'.format(i)))
x = Array(np.reshape(np.array(x),(1,row * column)))

#Starting point of arm
start = [0,0]

#Weight(distance of movement)
w0 = []
w = [[] for i in range(column)]
wf = []

for i in range(column):
    w0.append(dist_cal(start, tasks[i]))
    for j in range(column):
        if i % 2 == 0:
            w[i].append(dist_cal(tasks[i + 1], tasks[j]))
        else:
            w[i].append(dist_cal(tasks[i - 1], tasks[j]))
    if i % 2 == 0:
        wf.append(dist_cal(tasks[i + 1], start))
    else:
        wf.append(dist_cal(tasks[i - 1], start))
w0 = np.array(w0)
w = np.array(w)
wf = np.array(wf)

p = []

for i in range(row):
    p.append(max(np.amax(w[:n,i * 2:(i + 1) * 2]),max(w0[i * 2:(i + 1) * 2]))+np.amax(w[i * 2 : (i + 1) * 2]))

p = p / max(p)

Pt = max(p)

#Cost function
H_dists = sum(x[0, :column] * w0)
for t in range(1, row):
    for i in range(column):
        H_dists += sum(np.multiply(w[i] * x[0, column * t: column * (t + 1)], x[0, column * (t - 1) + i]))
H_dists += sum(x[0, column * (row - 1): column * row] * wf)

#Constraint-1
H_tasks = 0
for i in range(row):
    s = 0
    for j in range(row):
        s += x[0, column * j + (i * 2)] + x[0, column * j + (i * 2) + 1]
    H_tasks += p[i] * ((s - 1) ** 2)
H_tasks = Constraint(H_tasks, "tasks")

#Constraint-2
H_time = 0
for i in range(row):
    H_time += Pt * ((sum(x[0, i * column: (i + 1) * column]) - 1) ** 2)
H_time = Constraint(H_time, "time")

H_cost = H_dists + Placeholder("tasks") * H_tasks + Placeholder("time") * H_time
model = H_cost.compile()
#Weigth for Constrain
para = {}
for k in range(5,6):
    for l in range(6500,8000):
        feed_dict = {'tasks': 250, 'time': l * 0.1}
        qubo,offset = model.to_qubo(feed_dict=feed_dict)

        Q = {(int(re.search(r"x\[([0-9]+)\]", i)[1]),
               int(re.search(r"x\[([0-9]+)\]", j)[1])): v for (i, j), v in qubo.items()}

        sampler = neal.SimulatedAnnealingSampler()
        ans = []
        # Sampling by D-Wave
        result = sampler.sample_qubo(Q, num_reads = 10000,num_sweeps = 1000,beta_schedule_type= 'linear')
        np.set_printoptions(threshold=100000000)
        i = 0
        j = 0
        for sample in result:
            if [k for k, v in sample.items() if v == 1] == [3, 6, 17]:
                i += 1
            if [k for k, v in sample.items() if v == 1] == [4, 7, 14]:
                j += 1
        para[(k,l)] = i + j
print('finished')

finished


In [70]:
pprint.pprint(para)

{(5, 5): 0,
 (5, 6): 0,
 (5, 7): 0,
 (5, 8): 0,
 (5, 9): 0,
 (5, 10): 0,
 (5, 11): 0,
 (5, 12): 1,
 (5, 13): 1532,
 (5, 14): 3962,
 (5, 15): 3953,
 (5, 16): 3447,
 (5, 17): 3063,
 (5, 18): 2780,
 (5, 19): 2420,
 (5, 20): 2267,
 (6, 5): 0,
 (6, 6): 0,
 (6, 7): 0,
 (6, 8): 0,
 (6, 9): 0,
 (6, 10): 0,
 (6, 11): 1,
 (6, 12): 1443,
 (6, 13): 3748,
 (6, 14): 3681,
 (6, 15): 3544,
 (6, 16): 3598,
 (6, 17): 3487,
 (6, 18): 3392,
 (6, 19): 3261,
 (6, 20): 2944,
 (7, 5): 0,
 (7, 6): 0,
 (7, 7): 0,
 (7, 8): 0,
 (7, 9): 0,
 (7, 10): 0,
 (7, 11): 1404,
 (7, 12): 3577,
 (7, 13): 3499,
 (7, 14): 3508,
 (7, 15): 3416,
 (7, 16): 3340,
 (7, 17): 3440,
 (7, 18): 3271,
 (7, 19): 3296,
 (7, 20): 3284,
 (8, 5): 0,
 (8, 6): 0,
 (8, 7): 0,
 (8, 8): 0,
 (8, 9): 1,
 (8, 10): 1530,
 (8, 11): 3551,
 (8, 12): 3459,
 (8, 13): 3422,
 (8, 14): 3352,
 (8, 15): 3213,
 (8, 16): 3178,
 (8, 17): 3174,
 (8, 18): 3170,
 (8, 19): 3054,
 (8, 20): 3043,
 (9, 5): 0,
 (9, 6): 0,
 (9, 7): 0,
 (9, 8): 1,
 (9, 9): 1818,
 (9, 10): 3

In [76]:
pprint.pprint(para)

{(5, 6500): 1477,
 (5, 6501): 1529,
 (5, 6502): 1591,
 (5, 6503): 1533,
 (5, 6504): 1599,
 (5, 6505): 1592,
 (5, 6506): 1639,
 (5, 6507): 1644,
 (5, 6508): 1684,
 (5, 6509): 1696,
 (5, 6510): 1634,
 (5, 6511): 1721,
 (5, 6512): 1719,
 (5, 6513): 1791,
 (5, 6514): 1807,
 (5, 6515): 1804,
 (5, 6516): 1863,
 (5, 6517): 1752,
 (5, 6518): 1839,
 (5, 6519): 1958,
 (5, 6520): 1874,
 (5, 6521): 1884,
 (5, 6522): 1920,
 (5, 6523): 1986,
 (5, 6524): 2032,
 (5, 6525): 2006,
 (5, 6526): 2045,
 (5, 6527): 2109,
 (5, 6528): 2003,
 (5, 6529): 2091,
 (5, 6530): 2088,
 (5, 6531): 2116,
 (5, 6532): 2118,
 (5, 6533): 2199,
 (5, 6534): 2260,
 (5, 6535): 2215,
 (5, 6536): 2261,
 (5, 6537): 2178,
 (5, 6538): 2226,
 (5, 6539): 2291,
 (5, 6540): 2333,
 (5, 6541): 2318,
 (5, 6542): 2338,
 (5, 6543): 2325,
 (5, 6544): 2439,
 (5, 6545): 2357,
 (5, 6546): 2392,
 (5, 6547): 2465,
 (5, 6548): 2484,
 (5, 6549): 2480,
 (5, 6550): 2534,
 (5, 6551): 2592,
 (5, 6552): 2580,
 (5, 6553): 2576,
 (5, 6554): 2555,
 (5, 6555)